In [63]:
import pandas as pd
import ibm_db_dbi
import ibm_db
# from sqlalchemy import create_engine

In [64]:
dsn_hostname = "1bbf73c5-d84a-4bb0-85b9-ab1a4348f4a4.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud"
dsn_uid = "xkk84284"        # e.g. "abc12345"
dsn_pwd = "QdRxktEwkW0eOMFE"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "bludb"            # e.g. "BLUDB"
dsn_port = "32286"                # e.g. "32733"
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"          #i.e. "SSL"

In [4]:
# Create database connection
# DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  bludb as user:  xkk84284 on host:  1bbf73c5-d84a-4bb0-85b9-ab1a4348f4a4.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud


In [91]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)
conn = ibm_db.connect(dsn, "", "")
pconn = ibm_db_dbi.Connection(conn)

In [197]:
df = pd.read_sql('SELECT * FROM CHICAGO',pconn)
sum(df["COLLEGE_ELIGIBILITY__"] == "NDA")

480

In [253]:
query="""
  SELECT 
    COMMUNITY_AREA_NAME,
    COMMUNITY_AREA_NUMBER,
    COLLEGE_ELIGIBILITY__,
    NAME_OF_SCHOOL
  FROM CHICAGO
  WHERE COLLEGE_ELIGIBILITY__ NOT LIKE \'NDA\'
  ORDER BY 
    COMMUNITY_AREA_NUMBER,
    CAST(COLLEGE_ELIGIBILITY__ AS INT) DESC
"""
df = pd.read_sql(query, pconn)
df.tail(20)

,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,COLLEGE_ELIGIBILITY__,NAME_OF_SCHOOL
66,ARCHER HEIGHTS,57,22.9,Marie Sklodowska Curie Metropolitan High School
67,BRIGHTON PARK,58,22.1,Thomas Kelly High School
68,NEW CITY,61,10.5,Edward Tilden Career Community Academy High Sc...
69,NEW CITY,61,5.5,Ellen H Richards Career Academy High School
70,WEST ELSDON,62,22.6,John Hancock College Preparatory High School
71,GAGE PARK,63,7.9,Gage Park High School
72,WEST LAWN,65,30.1,Gurdon S Hubbard High School
73,WEST ENGLEWOOD,67,40,Southside Occupational Academy High School
74,WEST ENGLEWOOD,67,29.1,Robert Lindblom Math & Science Academy High Sc...
75,WEST ENGLEWOOD,67,5.9,William Rainey Harper High School


In [147]:
cols = ["SCHOOL_ID", "COLLEGE_ELIGIBILITY__"]
df2 = df[cols]
df2

KeyError: "['COLLEGE_ELIGIBILITY__'] not in index"

# Question 1. Calculate the total number of students in each grade level.

In [20]:
q1 = pd.read_sql(
  """
  SELECT
      ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL,
      COUNT(*) as total_number_of_students
  FROM chicago
  GROUP BY ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL;
  """,
pconn)
q1

,ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL,TOTAL_NUMBER_OF_STUDENTS
0,ES,462
1,HS,93
2,MS,11


# QUESTION 2.Find the top 5 student groups with the largest number of Students and highest safety

In [21]:
q2 = pd.read_sql(
  """
  SELECT
    NAME_OF_SCHOOL,SAFETY_SCORE, COLLEGE_ENROLLMENT, SAFETY_ICON
  FROM chicago
  WHERE SAFETY_SCORE not like \'%NONE%\'
  ORDER BY SAFETY_SCORE DESC, COLLEGE_ENROLLMENT DESC LIMIT 5
  """
  ,pconn)
q2

,NAME_OF_SCHOOL,SAFETY_SCORE,COLLEGE_ENROLLMENT,SAFETY_ICON
0,Northside College Preparatory High School,99,1053,Very Strong
1,Alexander Graham Bell Elementary School,99,998,Very Strong
2,James G Blaine Elementary School,99,923,Very Strong
3,Abraham Lincoln Elementary School,99,813,Very Strong
4,Oriole Park Elementary School,99,648,Very Strong


In [22]:
df = pd.read_sql('SELECT * FROM CHICAGO',pconn)

col = ["NAME_OF_SCHOOL", "SAFETY_SCORE", "COLLEGE_ENROLLMENT", "SAFETY_ICON"]
df[col].sort_values(["SAFETY_SCORE", "COLLEGE_ENROLLMENT"], ascending=False).head(5)

,NAME_OF_SCHOOL,SAFETY_SCORE,COLLEGE_ENROLLMENT,SAFETY_ICON
408,Northside College Preparatory High School,99.0,1053,Very Strong
10,Alexander Graham Bell Elementary School,99.0,998,Very Strong
245,James G Blaine Elementary School,99.0,923,Very Strong
0,Abraham Lincoln Elementary School,99.0,813,Very Strong
417,Oriole Park Elementary School,99.0,648,Very Strong


# Question 3. Calculate average of AVERAGE_STUDENT_ATTENDANCE of 10 schools with least SAFETY_SCORE

In [23]:
q3 = pd.read_sql(
    """
    SELECT ROUND(AVG(CAST(REPLACE(AVERAGE_STUDENT_ATTENDANCE, '%', '') AS DECIMAL(5, 2))),2) AS average_attendance
    FROM (
        SELECT 
            NAME_OF_SCHOOL,
            AVERAGE_STUDENT_ATTENDANCE,
            SAFETY_SCORE
        FROM chicago
        ORDER BY SAFETY_SCORE
        LIMIT 10
    ) AS subquery;
    """, 
    pconn
)
q3

,AVERAGE_ATTENDANCE
0,89.12


In [42]:
df = pd.read_sql("""SELECT 
            Name_OF_SCHOOL,
            AVERAGE_STUDENT_ATTENDANCE,
            SAFETY_SCORE
        FROM chicago
        ORDER BY SAFETY_SCORE
        LIMIT 10""",
  pconn)
df

,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE,SAFETY_SCORE
0,Edmond Burke Elementary School,89.30%,1
1,Luke O'Toole Elementary School,89.20%,5
2,George W Tilton Elementary School,91.40%,6
3,Foster Park Elementary School,93.70%,11
4,Emil G Hirsch Metropolitan High School,84.80%,13
5,William W Carter Elementary School,91.20%,13
6,John Fiske Elementary School,92.90%,13
7,Countee Cullen Elementary School,94.10%,14
8,Gage Park High School,72.50%,14
9,Asa Philip Randolph Elementary School,92.10%,15


# Question 4. Find 10 High school with highest College_Eligibility__ score.

In [24]:
q4 = pd.read_sql(
    """
    SELECT Name_OF_SCHOOL, CAST(College_Eligibility__ AS FLOAT) AS College_Eligibility_Score
    FROM chicago
    WHERE COLLEGE_ELIGIBILITY__ NOT LIKE '%NDA%'
    ORDER BY College_Eligibility_Score DESC
    LIMIT 10;
    """,
    pconn
)
q4

,NAME_OF_SCHOOL,COLLEGE_ELIGIBILITY_SCORE
0,Northside College Preparatory High School,98.0
1,Walter Payton College Preparatory High School,96.4
2,Whitney M Young Magnet High School,89.0
3,William Jones College Preparatory High School,83.3
4,Albert G Lane Technical High School,67.9
5,Northside Learning Center High School,66.7
6,Jacqueline B Vaughn Occupational High School,55.6
7,Lincoln Park High School,53.6
8,Gwendolyn Brooks College Preparatory Academy H...,51.1
9,DeVry University Advantage Academy High School,40.6


# CHICAGO MAP

In [ ]:
! pip install openpyxl

In [ ]:
! mamba install openpyxl==3.0.9 -y

In [ ]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [ ]:
! pip3 install folium==0.5.0
import folium

print('Folium installed and imported!')

In [ ]:
# define the world map
world_map = folium.Map()

# display world map
world_map

In [ ]:
# define the Chicago map with a higher zoom level
chicago_map = folium.Map(location=[41.84969736, -87.6252787], zoom_start=12)

# display Chicago map
chicago_map

#save Chicago map into .html
chicago_map.save('chicago.html')

In [ ]:
folium.Marker([42.01017628, -87.68446868])

#### Figure out all school destination

In [ ]:
import folium

print('Folium installed and imported!')

In [ ]:
# define the world map
world_map = folium.Map()

# display world map
world_map

In [ ]:
loc = pd.read_sql('SELECT REPLACE(REPLACE(Location, \'(\', \'\'), \')\', \'\') as des FROM CHICAGO', pconn)
loc

In [ ]:
kd = pd.read_sql('SELECT Latitude FROM Chicago', pconn)
vd = pd.read_sql('SELECT Longitude FROM Chicago', pconn)

In [ ]:
sl = """SELECT * FROM CHICAGO"""
df = pd.read_sql(sl,pconn)
from folium.plugins import MarkerCluster
chicago_map = folium.Map(location=[41.84969736, -87.6252787], zoom_start=12)


tooltip = 'Click me!'


df1 = df[['LATITUDE','LONGITUDE','NAME_OF_SCHOOL']]
# df2 = df[['LATITUDE','LONGITUDE']]
marker_cluster = MarkerCluster().add_to(chicago_map)
for index, row in df1.iterrows():
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    label = row['NAME_OF_SCHOOL']
    folium.Marker([latitude, longitude],
                popup=folium.Popup(label,parse_html=True,max_width='1000000%'),
                icon=folium.Icon(icon='glyphicon glyphicon-headphones',)).add_to(marker_cluster)

chicago_map.save('chicathanh.html')
chicago_map

In [ ]:
import folium
import pandas as pd

# Define the Chicago map with a higher zoom level
chicago_map = folium.Map(location=[41.84969736, -87.6252787], zoom_start=12)

# Assuming you have DataFrames 'kd' and 'vd' with latitude and longitude data
# Replace 'LATITUDE' and 'LONGITUDE' with your actual column names
url = "https://leafletjs.com/examples/custom-icons/{}".format
icon_image = url("leaf-red.png")
shadow_image = url("leaf-shadow.png")

icon = folium.CustomIcon(
    icon_image,
    icon_size=(38, 95),
    icon_anchor=(22, 94),
    shadow_image=shadow_image,
    shadow_size=(50, 64),
    shadow_anchor=(4, 62),
    popup_anchor=(-3, -76),
)
# Iterate through rows in 'kd' and 'vd' DataFrames and add markers
for index, row in kd.iterrows():
    latitude = row['LATITUDE']  # Replace 'LATITUDE' with your actual column name
    longitude = vd.iloc[index]['LONGITUDE']  # Replace 'LONGITUDE' with your actual column name

    # Create markers with latitude and longitude
    folium.Marker(location=[latitude, longitude]).add_to(chicago_map)

# Save the Chicago map to an HTML file
chicago_map.save('chicago.html')

# Display the Chicago map (if running in a Jupyter Notebook or interactive environment)
chicago_map


In [ ]:
ibm_db.close()